## This notebook demonstrates how to provide a custom stamp for ghost simulation in Mirage.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import yaml
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
from glob import glob
import jwst

from astropy.io import fits,ascii
from astropy.modeling import models, fitting
from astropy.table import QTable, Table, Column


In [ ]:
# Deconstruct;
from NIRCAM_Gsim.grism_seed_disperser import Grism_seed
from mirage.seed_image import catalog_seed_image
from mirage.yaml import yaml_generator
from mirage import imaging_simulator
from mirage.ramp_generator import obs_generator
from mirage.dark import dark_prep


In [ ]:
jwst.__version__

In [ ]:
import mirage
mirage.__version__

In [ ]:
# Get yaml file;
# Both GCANUCS and GLASS are Medium dither pattern.
xml_file      = './files/1085.xml' 
pointing_file = './files/1085.pointing'

# Yaml output directory;
yaml_output_dir = './yaml_custom/'
if not os.path.exists(yaml_output_dir):
    os.mkdir(yaml_output_dir)
    
# Setup miscs:
file_point_name = './files/stars_example.txt' #'/ifs/jwst/wit/niriss/kevin/new_focus_field/stars_new_focus_field_allfilters.txt'
catalogs = {'point_source': file_point_name}

reffile_defaults = 'crds'
cosmic_rays = {'library': 'SUNMAX', 'scale': 1.0}
dates = '2022-07-01'
pav3 = 0
background = 'low'
simulations_output_dir = 'output_custom'
datatype = 'raw'


In [ ]:
# Your postage stamp for ghost;
file_stamp_gst = './files/ghost_stamp.fits'


In [ ]:
## Tweak extended catalog, to switch ghost stamp to an arbitrary one;

file_point_name_new = './files/stars_example_new.txt' #'/ifs/jwst/wit/niriss/kevin/new_focus_field/stars_new_focus_field_allfilters.txt'
fd = ascii.read(file_point_name)
ra = fd['x_or_RA']
dec = fd['y_or_Dec']

source_stamp_files = []
for ii in range(len(ra)):
    source_stamp_files.append(file_stamp_gst)

from mirage.catalogs import catalog_generator as cg
cat = cg.PointSourceCatalog(ra=ra, dec=dec, niriss_ghost_stamp=source_stamp_files)

niriss_filters = ['f090w','f115w','f150w','f200w']
for filt in niriss_filters:
    niriss_magnitude = fd['niriss_%s_magnitude'%filt]
    cat.add_magnitude_column(niriss_magnitude, instrument='niriss', filter_name='%s'%filt.upper(), magnitude_system='vegamag')
cat.save(file_point_name_new)

catalogs = {'point_source': file_point_name_new}


In [ ]:
# Run the yaml generator
yam = yaml_generator.SimInput(input_xml=xml_file, pointing_file=pointing_file,
                              catalogs=catalogs, cosmic_rays=cosmic_rays,
                              background=background,roll_angle=pav3,
                              dates=dates, reffile_defaults=reffile_defaults,
                              verbose=True, output_dir=yaml_output_dir,
                              simdata_output_dir=simulations_output_dir,
                              datatype=datatype,add_ghosts=True)

yam.create_inputs()

In [ ]:
# This is output directory;
DIR_OUT = './output_custom/'
if not os.path.exists(DIR_OUT):
    os.mkdir(DIR_OUT)


# Run simulator

In [ ]:
pfiles = glob('%s/*01101_*_nis.yaml'%yaml_output_dir)
pfiles

### Create scene:

In [ ]:
yaml_WFSS_files = []
yaml_NIRISS_imaging_files = []

for f in pfiles:
    my_dict = yaml.safe_load(open(f))
    if my_dict["Inst"]["mode"]=="wfss":
        yaml_WFSS_files.append(f)
    if my_dict["Inst"]["mode"]=="imaging" and my_dict['Inst']['instrument']=='NIRISS':
        yaml_NIRISS_imaging_files.append(f)
    
print("WFSS files:",len(yaml_WFSS_files))
print("Imaging files:",len(yaml_NIRISS_imaging_files))

yaml_NIRISS_imaging_files

In [ ]:
# Direct images;
for file in yaml_NIRISS_imaging_files[:1]:
    img_sim = imaging_simulator.ImgSim()
    img_sim.paramfile = file
    img_sim.create()
    

In [ ]:
# WFSS;
from mirage import wfss_simulator

for file in yaml_WFSS_files[:1]:
    m = wfss_simulator.WFSSSim(file, override_dark=None, save_dispersed_seed=True,
                               extrapolate_SED=None, disp_seed_filename=None, source_stamps_file=None,
                               SED_file=None, SED_normalizing_catalog_column=None, SED_dict=None,
                               create_continuum_seds=True)
    
    m.create()


In [ ]:
file_seed = 'output_custom/jw01085001001_01101_00001_nis_uncal_CLEAR_F150W_final_seed_image.fits'
fd = fits.open(file_seed)[1].data

gaps = [1162.600, 937.900]

plt.plot(gaps[0], gaps[1], marker='x', color='orange', ms=10)
plt.imshow(fd, vmin=.1, vmax=3)
plt.xlim(850,1450)
plt.ylim(620,1220)

plt.savefig('ghost_custom_demo.png', dpi=200)
